In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext lab_black

In [2]:
from collections import defaultdict
import numpy as np
import random
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from dataset import load_dataset_from_path
from datasets import load_dataset, Dataset
from utils import format_query

/cluster/home/kevidu/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cluster/home/kevidu/venv/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

In [4]:
# ROOT_DATA_DIR = "../data/Yago/"
# RAW_DATA_PATH = os.path.join(ROOT_DATA_DIR, "yago_qec.json")

In [5]:
# Filtered to be only the entities/queries Llama2chat can answer correctly
ROOT_DATA_DIR = "../data/YagoLlama2/"
RAW_DATA_PATH = os.path.join(ROOT_DATA_DIR, "llama2chat_yago_qec.json")

In [6]:
dataset = load_dataset_from_path(RAW_DATA_PATH)
len(dataset.keys())
# dataset[:1]

66

In [7]:
list(dataset[list(dataset.keys())[1]].keys())

['answer_uris',
 'answers',
 'context_templates',
 'entities',
 'entity_namesake_to_degree',
 'entity_namesake_to_num_uris',
 'entity_uri_to_degree',
 'entity_uri_to_predicate_degree',
 'entity_uris',
 'gpt_fake_entities',
 'query_forms']

In [8]:
from tqdm import tqdm

In [11]:
my_dataset = defaultdict(list)
num_fake_contexts_per_query = 10
num_entities = 50
context_types = ["base"]
for query_id, qec in tqdm(dataset.items()):
    ents_and_answers = list(zip(qec["entities"], qec["answers"]))
    random.shuffle(ents_and_answers)
    ents_and_answers = ents_and_answers[:num_entities]
    for entity, answer in ents_and_answers:
        for qt, qfs in qec["query_forms"].items():
            for qf in qfs:
                if not qf.startswith("Q:"):
                    query = format_query(
                        query=qf, entity=(entity,), context="", answer=answer
                    )
                    for context_type in context_types:
                        ctx_template = qec["context_templates"][context_type]
                        fake_answers_all = list(set(qec["answers"]) - {answer})
                        fake_answers = random.sample(fake_answers_all, k=min(num_fake_contexts_per_query, len(fake_answers_all))) # min in case the query has fewer possible answers
                        for fake_answer in fake_answers: 
                            context = ctx_template.format(
                                entity=entity, answer=fake_answer
                            ).strip()
                            # add fake
                            my_dataset["context"] += [context]
                            my_dataset["query"] += [query]
                            my_dataset["weight_context"] += [1.0]
                            my_dataset["answer"] += [
                                fake_answer if qt == "open" else "No"
                            ]

                            # add real
                            my_dataset["context"] += [context]
                            my_dataset["query"] += [query]
                            my_dataset["weight_context"] += [0.0]
                            my_dataset["answer"] += [answer if qt == "open" else "Yes"]

                            # Add metadata shared between both examples
                            my_dataset["entity"] += [entity] * 2
                            my_dataset["ctx_answer"] += [fake_answer] * 2
                            my_dataset["prior_answer"] += [answer] * 2
                            my_dataset["query_id"] += [query_id] * 2
                            my_dataset["query_type"] += [qt] * 2


df_all = pd.DataFrame.from_dict(my_dataset)
df_all

100%|██████████| 66/66 [00:00<00:00, 560.47it/s]


,context,query,weight_context,answer,entity,ctx_answer,prior_answer,query_id,query_type
0,'Norma Talmadge filmography' is about Paul Anka.,'Norma Talmadge filmography' is about,1.0,Paul Anka,Norma Talmadge filmography,Paul Anka,Norma Talmadge,http://schema.org/about,open
1,'Norma Talmadge filmography' is about Paul Anka.,'Norma Talmadge filmography' is about,0.0,Norma Talmadge,Norma Talmadge filmography,Paul Anka,Norma Talmadge,http://schema.org/about,open
2,'Norma Talmadge filmography' is about Morgan F...,'Norma Talmadge filmography' is about,1.0,Morgan Freeman,Norma Talmadge filmography,Morgan Freeman,Norma Talmadge,http://schema.org/about,open
3,'Norma Talmadge filmography' is about Morgan F...,'Norma Talmadge filmography' is about,0.0,Norma Talmadge,Norma Talmadge filmography,Morgan Freeman,Norma Talmadge,http://schema.org/about,open
4,'Norma Talmadge filmography' is about Shilpa S...,'Norma Talmadge filmography' is about,1.0,Shilpa Shetty,Norma Talmadge filmography,Shilpa Shetty,Norma Talmadge,http://schema.org/about,open
...,...,...,...,...,...,...,...,...,...
27673,Öland County was replaced by Kagoshima.,Öland County was replaced by,0.0,Kalmar County,Öland County,Kagoshima,Kalmar County,reverse-http://yago-knowledge.org/resource/rep...,open
27674,A student of Charles Émile Picard was Jacques ...,A student of Charles Émile Picard was,1.0,Jacques Hadamard,Charles Émile Picard,Jacques Hadamard,Paul Painlevé,reverse-http://yago-knowledge.org/resource/stu...,open
27675,A student of Charles Émile Picard was Jacques ...,A student of Charles Émile Picard was,0.0,Paul Painlevé,Charles Émile Picard,Jacques Hadamard,Paul Painlevé,reverse-http://yago-knowledge.org/resource/stu...,open
27676,A student of Charles Émile Picard was Paul Pai...,A student of Charles Émile Picard was,1.0,Paul Painlevé,Charles Émile Picard,Paul Painlevé,Jacques Hadamard,reverse-http://yago-knowledge.org/resource/stu...,open


In [12]:
# Design choice: since we don't foresee needing to change the train/val/test fractions much, we just produce CSVs (albeit somewhat low-provenance) in this script.
# If we wanted to be able to vary train/val/test fractions for some reason (e.g. Flatiron needed to to balance training and test set sizes for different diseases, e.g. in a pan-tumor model), then we should be more careful about parameterizing the train/val/test fracs.
from typing import List


def tuple_df(df):
    return list(df.itertuples(index=False, name=None))


def partition_df(df, columns: List[str], val_frac=0.2, test_frac=0.2):
    keys_df = df[columns].drop_duplicates()
    train_keys_df, test_keys_df = train_test_split(
        keys_df, test_size=test_frac, random_state=SEED
    )
    train_keys_df, val_keys_df = train_test_split(
        train_keys_df, test_size=val_frac, random_state=SEED
    )

    train_df = df_all.merge(train_keys_df, on=columns, how="inner")
    val_df = df_all.merge(val_keys_df, on=columns, how="inner")
    test_df = df_all.merge(test_keys_df, on=columns, how="inner")

    assert len(train_df) + len(val_df) + len(test_df) == len(df_all)
    assert not set(tuple_df(train_df[columns])).intersection(tuple_df(val_df[columns]))
    assert not set(tuple_df(train_df[columns])).intersection(tuple_df(test_df[columns]))

    return train_df, val_df, test_df


# COLS = ["rel_p_id"]
# train_df, val_df, test_df = partition_df(df_all, COLS)

In [13]:
dir_to_cols = {
    "nodup_relpid": ["query_id"],
    # "nodup_relpid_subj": ["rel_p_id", "subject"],
    # "nodup_relpid_obj": ["rel_p_id", "object"],
    # "base": ["subject", "rel_p_id", "object"],
}

for dir, cols in dir_to_cols.items():
    full_dir = os.path.join(ROOT_DATA_DIR, "splits", dir)
    os.makedirs(full_dir, exist_ok=True)
    train_df, val_df, test_df = partition_df(df_all, cols)
    train_df.sample(frac=1).reset_index(drop=True).to_csv(os.path.join(full_dir, "train.csv"), index=False)
    val_df.sample(frac=1).reset_index(drop=True).to_csv(os.path.join(full_dir, "val.csv"), index=False)
    test_df.sample(frac=1).reset_index(drop=True).to_csv(os.path.join(full_dir, "test.csv"), index=False)

In [17]:
print(train_df["query_id"].unique(), len(train_df["query_id"].unique()))
print(val_df["query_id"].unique(), len(val_df["query_id"].unique()))
print(test_df["query_id"].unique(), len(test_df["query_id"].unique()))

['http://schema.org/about' 'http://schema.org/address'
 'http://schema.org/author' 'http://schema.org/children'
 'http://schema.org/demonym' 'http://schema.org/director'
 'http://schema.org/duration' 'http://schema.org/elevation'
 'http://schema.org/founder' 'http://schema.org/highestPoint'
 'http://schema.org/homeLocation' 'http://schema.org/icaoCode'
 'http://schema.org/inLanguage' 'http://schema.org/influencedBy'
 'http://schema.org/manufacturer' 'http://schema.org/material'
 'http://schema.org/nationality' 'http://schema.org/numberOfPages'
 'http://schema.org/officialLanguage' 'http://schema.org/organizer'
 'http://schema.org/ownedBy' 'http://schema.org/parentTaxon'
 'http://schema.org/postalCode' 'http://schema.org/sponsor'
 'http://schema.org/unemploymentRate'
 'http://yago-knowledge.org/resource/flowsInto'
 'http://yago-knowledge.org/resource/parentBody'
 'http://yago-knowledge.org/resource/participant'
 'http://yago-knowledge.org/resource/playsIn'
 'http://yago-knowledge.org/re